In [ ]:
# for Table 3, include all but Lasso and Elastic net
# k = 1 cpu = 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from itertools import compress
import time

from sklearn.linear_model import lasso_path, enet_path, LogisticRegression, ElasticNet, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR, LinearSVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE, SequentialFeatureSelector, SelectFromModel
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif, r_regression

import scipy.stats as ss

from helpers import expr_data
from helpers import scale_data
from helpers import similarity

import warnings
warnings.filterwarnings("ignore")

In [3]:
import random
np.random.seed(42)
random.seed(42)

## Load all data

In [5]:
data = expr_data.ExprData()
data.load_pickle()
data = data.fix_tpch()

#### Split by SKU

In [6]:
data_by_sku = data.split_by_sku()

#### Calculate Distance

In [ ]:
# the result sku_result is a dict with its key the SKU,
# the value a list, the classification accuracy for each f_num
data_dist = {}

for sku in data_by_sku.keys():
    curr_data = data_by_sku[sku]
    if 'ter' in sku:
        continue
    scaler = scale_data.ScaleData()
    plan_mtxs_splitted, plan_col_ranges = scaler.scale(curr_data.plan_mtxs)
    perf_mtxs_splitted, perf_col_ranges = scaler.scale(curr_data.perf_mtxs)
    simi_calc = similarity.Similarity(curr_data, plan_mtxs_splitted, plan_col_ranges, perf_mtxs_splitted, perf_col_ranges, num_bins=10)

    simi_calc.calc_bined_mtx() # all features
    simi_calc.calc_dist_simi_matrix(normalize=True)
    print(simi_calc.simi_mtx.shape)
    # feature wise distance
    simi_calc.calc_featurewise_dist_by_col()
    
    data_dist[sku] = simi_calc

## Select Top K Features

In [9]:
# return non-zero index in descending order
def sparse_argsort(arr):
    arr = np.where(np.isnan(arr), 0, arr)
    arr = arr * -1
    indices = np.nonzero(arr)[0]
    result = indices[np.argsort(arr[indices])]
    return result

def all_argsort(arr):
    arr = np.where(np.isnan(arr), 0, arr)
    arr = arr * -1
    result = np.argsort(arr)
    return result

### Filter Based

#### Variance threshold

In [10]:
def variance_threshold(X, y):
    selector = VarianceThreshold()
    selector.fit(X)
    scores = selector.variances_
    return sparse_argsort(scores)

#### fANOVA, Chi-Squared test, Mutual Information gain, Fisher score

In [11]:
# fANOVA, Chi-Squared test, Mutual Information gain, Fisher score
def select_k_best(X, y, method):
    if method == 'fANOVA':
        selector = SelectKBest(f_classif, k='all')
    elif method == 'Chi2':
        selector = SelectKBest(chi2, k='all')
    elif method == 'MutualInfoGain': # this uses knn=3 by default to do the feature selection 
        selector = SelectKBest(mutual_info_classif, k='all')  
    elif method == 'Pearson': #https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.r_regression.htm
        selector = SelectKBest(r_regression, k='all')  
    selector.fit(X, y)
    scores = np.absolute(selector.scores_)
    return sparse_argsort(scores)

### Wrapper method

#### RFE (Recursive feature elimination)

In [13]:
def get_est(est_name):
    if est_name == 'DecisionTree':
        estimator = DecisionTreeClassifier(criterion='entropy', max_depth=None)
    elif est_name == 'LogisticRegression':
        estimator = LogisticRegression(n_jobs=4, C=0.01) # C, tol, 
    else: # est_name == 'Linear':
        estimator = SVR(kernel="linear", C=0.05, ) # kernel, degree, 
    return estimator

In [14]:
def rfe_orders(X, y, est_name):
    estimator = get_est(est_name)
    selector = RFE(estimator, n_features_to_select=1, step=1)
    selector = selector.fit(X, y)
    # ranks are: 1 for most important
    scores = -1 * selector.ranking_
    return sparse_argsort(scores)

#### SFS (Sequential Feature Selection)

In [15]:
def sfs_orders(curr_data, direction, est_name, n):
    num_features = len(curr_data.feature_cols)
    feature_importance = np.array([0]*num_features)
    expr_num = curr_data.get_num_exprs()

    for i in range(expr_num):
        # calculate label
        curr_name = curr_data.wl_names[i]
        y = [curr_name == name for name in curr_data.wl_names]
        X = simi_calc.simi_col_mtx[i]
        estimator = get_est(est_name)

        selector = SequentialFeatureSelector(estimator, direction=direction.lower(), n_features_to_select=n, n_jobs=-1, cv=3)
        selector = selector.fit(X, y)
        mask = selector.get_support()
        for idx in range(num_features):
            feature_importance[idx] += mask[idx]
    final_orders = sparse_argsort(feature_importance)[:n]
    top_features = [curr_data.feature_cols[j] for j in final_orders]
    return top_features

### Embedded method

#### Linear

In [17]:
# not used
def lasso_weights_orders(X, y):
    selector = SelectFromModel(estimator=LinearSVC(C=0.01, penalty="l1", dual=False)).fit(X, y) # C 
    scores = np.abs(selector.estimator_.coef_[0])
    return sparse_argsort(scores)

#### Ridge

In [18]:
# not used
def ridge_weights_orders(X, y):
    selector = SelectFromModel(estimator=LinearSVC(C=0.01, penalty="l2", dual=False)).fit(X, y)
    scores = np.abs(selector.estimator_.coef_[0])
    return sparse_argsort(scores)

### Feature select main function

In [19]:
def get_top_features(curr_data, expr_num, simi_calc, method, note=None):

    # create dict for all features
    num_features = len(curr_data.feature_cols)
    feature_importance = np.array([0]*num_features)
    
    for i in range(expr_num):
        # calculate label
        curr_name = curr_data.wl_names[i]
        y = [curr_name == name for name in curr_data.wl_names]
        # X = simi_calc.dist_by_col_cube[i]
        X = simi_calc.simi_col_mtx[i]
        
        mask = np.ones(X.shape[0], dtype=bool)  
        X = X[mask]#.reshape(-1, 1)
                
        if method == 'Lasso':
            orders = lasso_weights_orders(X, y)
        elif method == 'ENet':
            orders = enet_weights_orders(X, y)
        elif method == 'Variance':
            orders = variance_threshold(X, y)
        elif method == 'fANOVA':
            orders = select_k_best(X, y, method='fANOVA')
        elif method == 'Chi2':
            orders = select_k_best(X, y, method='Chi2')
        elif method == 'MutualInfoGain':
            orders = select_k_best(X, y, method='MutualInfoGain')
        elif method == 'Pearson':
            orders = select_k_best(X, y, method='Pearson')
        elif method == 'RFE':
            orders = rfe_orders(X, y, note)

        for idx in range(len(orders)):
            # from 0 to last idx of orders
            # the score = num_features - idx
            #   for a entry with feature_idx important order idx idx
            # the higher the order, the more the score
            feature_importance[orders[idx]] += num_features-idx
    final_orders = all_argsort(feature_importance)
    top_features = [curr_data.feature_cols[j] for j in final_orders]
    return top_features

## Compare Feature Selection with Similarity Calculation

#### Experiment Setup

In [20]:
main_dict = {}
time_dict = {}

In [22]:
all_features = data_by_sku[list(data_by_sku.keys())[0]].feature_cols
feature_num = len(all_features)

knn_thresholds = [1, 2, 3]
direct_methods = ['Variance', 'fANOVA', 'Chi2', 'MutualInfoGain', 'Pearson', 'Lasso', 'Ridge']
wrapper_methods = ['RFE']
estimator_names = ['Linear', 'DecisionTree', 'LogisticRegression']
other_methods = ['SFS', ]
simi_method = 'KNN'

f_nums = [1, 3, 7, 15, feature_num]

In [23]:
for knn_threshold in knn_thresholds:
    print(knn_threshold)
    
    if knn_threshold not in main_dict:
        main_dict[knn_threshold] = {}
        time_dict[knn_threshold] = {}
    for fs_method in direct_methods:
        print(fs_method)
        curr_method = {}

        for f_num in f_nums:
            curr_method[f_num] = []
        elapsed = []
        for sku in data_by_sku.keys():    
            if 'ter' in sku:
                continue
            curr_data = data_by_sku[sku]
            curr_calc = data_dist[sku]
            expr_num = curr_data.get_num_exprs()
        
            all_accs = []
            # run 10 times to get the average
            num_repeats = 5
            for i in range(num_repeats):       
                curr_accs = []
                start_time = time.time()
                top_features = get_top_features(curr_data, expr_num, curr_calc, fs_method, None)
                f_features = [top_features[:n] for n in f_nums]
                elapsed.append(time.time() - start_time)

                for f_num, curr_f in zip(f_nums, f_features):
                    curr_calc.calc_dist_simi_matrix(feature_names=curr_f)
                    pen, pens = curr_calc.simi_penalty(n=knn_threshold, dependent=True)

                    acc = 1 - (np.sum(pens)/(len(pens)*10))
                    curr_accs.append(acc)
                all_accs.append(curr_accs)
            all_accs = np.average(np.array(all_accs), axis=0)
            for f_num, acc in zip(f_nums, all_accs):
                curr_method[f_num].append(acc)
        main_dict[knn_threshold][fs_method] = curr_method
        time_dict[knn_threshold][fs_method] = np.mean(elapsed)
        print(np.mean(elapsed))

1
Variance
0.019920861721038817
fANOVA
0.03983873128890991
Chi2
0.062112510204315186
MutualInfoGain
2.4026515007019045
Pearson
0.02980543375015259
Lasso
0.04856536388397217
Ridge
0.05086199045181274
2
Variance
0.022982406616210937
fANOVA
0.04169530868530273
Chi2
0.06010161638259888
MutualInfoGain
2.2380417943000794
Pearson
0.028130996227264404
Lasso
0.048056495189666745
Ridge
0.04831134080886841
3
Variance
0.02095578908920288
fANOVA
0.039502954483032225
Chi2
0.06157795190811157
MutualInfoGain
2.2175882339477537
Pearson
0.029135119915008546
Lasso
0.04887100458145142
Ridge
0.05004221200942993


In [24]:
for knn_threshold in knn_thresholds:
    print(knn_threshold)
    
    if knn_threshold not in main_dict:
        main_dict[knn_threshold] = {}
        time_dict[knn_threshold] = {}

    for fs_method in wrapper_methods:
        for est_name in estimator_names:
            print(fs_method, est_name)
            curr_method = {}

            for f_num in f_nums:
                curr_method[f_num] = []
            elapsed = []
            for sku in data_by_sku.keys():    
                if 'ter' in sku:
                    continue
                curr_data = data_by_sku[sku]
                curr_calc = data_dist[sku]
                expr_num = curr_data.get_num_exprs()

                all_accs = []
                # run 10 times to get the average
                num_repeats = 5
                for i in range(num_repeats):       
                    curr_accs = []
                    start_time = time.time()
                    top_features = get_top_features(curr_data, expr_num, curr_calc, fs_method, est_name)
                    f_features = [top_features[:n] for n in f_nums]
                    elapsed.append(time.time() - start_time)
                    print(elapsed[-1])

                    for f_num, curr_f in zip(f_nums, f_features):
                        curr_calc.calc_dist_simi_matrix(feature_names=curr_f)
                        pen, pens = curr_calc.simi_penalty(n=knn_threshold)

                        acc = 1 - (np.sum(pens)/(len(pens)*10))
                        curr_accs.append(acc)
                    all_accs.append(curr_accs)
                all_accs = np.average(np.array(all_accs), axis=0)
                for f_num, acc in zip(f_nums, all_accs):
                    curr_method[f_num].append(acc)
            main_dict[knn_threshold][f'{fs_method}_{est_name}'] = curr_method
            time_dict[knn_threshold][f'{fs_method}_{est_name}'] = np.mean(elapsed)

1
RFE Linear
1.1427931785583496
1.2453176975250244
1.1666507720947266
1.1654529571533203
1.1564786434173584
1.0471961498260498
0.9480254650115967
0.8866641521453857
0.9317972660064697
0.99111008644104
1.1839320659637451
1.1457054615020752
1.1397418975830078
1.1389868259429932
1.2153856754302979
1.2917003631591797
1.1391184329986572
1.2313146591186523
1.2053768634796143
1.1971681118011475
RFE DecisionTree
1.2365655899047852
1.2155547142028809
1.167820692062378
1.2284789085388184
1.2097644805908203
1.1370701789855957
1.1596333980560303
1.1356406211853027
1.2304506301879883
1.2583816051483154
1.2438724040985107
1.2047629356384277
1.2223808765411377
1.2022883892059326
1.2112970352172852
1.1805405616760254
1.2138521671295166
1.262557029724121
1.230731725692749
1.0961802005767822
RFE LogisticRegression
34.02939224243164
17.643675804138184
17.653817653656006
17.38099479675293
17.844050407409668
17.882221698760986
17.878328323364258
18.089991092681885
18.076930284500122
17.687262535095215
18.3

In [25]:
for knn_threshold in knn_thresholds:
    print(knn_threshold)
    
    if knn_threshold not in main_dict:
        main_dict[knn_threshold] = {}
        time_dict[knn_threshold] = {}

    fs_method = 'SFS'
    for direction in ['Forward', 'Backward']:
        for est_name in estimator_names:
            print(direction+'_'+fs_method+'_'+est_name)
            curr_method = {}

            for f_num in f_nums:
                curr_method[f_num] = []
            elapsed = []
            for sku in data_by_sku.keys():    
                if 'ter' in sku:
                    continue
                curr_data = data_by_sku[sku]
                curr_calc = data_dist[sku]
                expr_num = curr_data.get_num_exprs()

                all_accs = []
                # run 10 times to get the average
                num_repeats = 1
                for i in range(num_repeats):       
                    curr_accs = []
                    start_time = time.time()
                    f_features = [sfs_orders(curr_data, direction, est_name, n) if n < feature_num else all_features for n in f_nums]
                    elapsed.append(time.time() - start_time)
                    print(elapsed[-1])

                    for f_num, curr_f in zip(f_nums, f_features):
                        curr_calc.calc_dist_simi_matrix(feature_names=curr_f)
                        pen, pens = curr_calc.simi_penalty(n=knn_threshold)

                        acc = 1 - (np.sum(pens)/(len(pens)*10))
                        curr_accs.append(acc)
                    all_accs.append(curr_accs)
                all_accs = np.average(np.array(all_accs), axis=0)
                for f_num, acc in zip(f_nums, all_accs):
                    curr_method[f_num].append(acc)
            main_dict[knn_threshold][direction+'_'+fs_method+'_'+est_name] = curr_method
            time_dict[knn_threshold][direction+'_'+fs_method+'_'+est_name] = np.mean(elapsed)

1
Forward_SFS_Linear
545.5392210483551
560.2021689414978
590.7735526561737
623.7616631984711
Forward_SFS_DecisionTree
675.3220210075378
719.2019350528717
734.5418281555176
759.2209367752075
Forward_SFS_LogisticRegression
1761.1950585842133
1817.3168053627014
1847.6125705242157
1892.8254897594452
Backward_SFS_Linear
2473.488553762436
2688.293385028839
2886.1261723041534
3127.849019050598
Backward_SFS_DecisionTree
3417.977571249008
3704.5509645938873
4065.6282699108124
4726.674874782562
Backward_SFS_LogisticRegression
7460.121118783951
8485.44242477417
14256.899272203445
15331.577209234238
2
Forward_SFS_Linear
7966.338711023331
15715.799492359161


KeyboardInterrupt: 

In [26]:
for key, val in main_dict.items():
    print(f'k = {key}')
    for subk, subval in val.items():
        print(f'method = {subk}')
        for fnum, subsubval in subval.items():
            print(fnum, subsubval)
# print(main_dict[knn_threshold])

k = 1
method = Variance
1 [0.4833333333333333, 0.24722222222222223, 0.24722222222222223, 0.25]
3 [0.7166666666666667, 0.7861111111111111, 0.625, 0.6805555555555556]
7 [0.9972222222222221, 0.9944444444444445, 0.9972222222222221, 0.9972222222222221]
15 [0.9972222222222221, 0.9944444444444445, 0.9972222222222221, 0.9972222222222221]
29 [0.9944444444444445, 0.9972222222222221, 0.9972222222222221, 0.9972222222222221]
method = fANOVA
1 [0.9694444444444444, 0.9472222222222222, 0.9527777777777777, 0.975]
3 [0.9833333333333332, 0.9444444444444444, 0.9555555555555555, 0.9944444444444445]
7 [0.986111111111111, 0.9527777777777777, 0.9805555555555555, 0.9916666666666668]
15 [0.9888888888888889, 0.9888888888888889, 0.9833333333333332, 0.9916666666666668]
29 [0.9944444444444445, 0.9972222222222221, 0.9972222222222221, 0.9972222222222221]
method = Chi2
1 [0.4833333333333333, 0.4833333333333333, 0.4833333333333333, 0.7305555555555556]
3 [0.9694444444444444, 0.95, 0.961111111111111, 0.7361111111111112]


In [28]:
def pretty_print_table(k=3, sku=None):
    name_trans_dict = {
        'Forward_SFS_Linear': 'Fw SFS Linear',
        'Backward_SFS_Linear': 'Bw SFS Linear',
        'Forward_SFS_DecisionTree': 'Fw SFS DecTree',
        'Backward_SFS_DecisionTree': 'Bw SFS DecTree',
        'Forward_SFS_LogisticRegression': 'Fw SFS LogReg',
        'Backward_SFS_LogisticRegression': 'Bw SFS LogReg',
        'MutualInfoGain': 'MIGain',
        'RFE_Linear': 'RFE Linear',
        'RFE_DecisionTree': 'RFE DecTree',
        'RFE_LogisticRegression': 'RFE LogReg',
    }
    
    sku_trans_dict = {
        'cpu2': 0,
        'cpu4': 1,
        'cpu8': 2,
        'cpu16': 3,
    }
    
    for method, subval in main_dict[k].items():
        outstr = '\\textcolor{}{'
        print_name = method if method not in name_trans_dict else name_trans_dict[method]
        outstr += print_name
        outstr += '} & '
        for fnum, subsubval in subval.items():
            if fnum == 29:
                continue
            if sku is None:
                outstr += f'{np.mean(subsubval):.3f} & '
            else:
                outstr += f'{subsubval[sku_trans_dict[sku]]:.3f} & '
        if method == 'Variance':
            if sku is None:
                all_acc = np.mean(subval[29])
            else:
                all_acc = subval[29][sku_trans_dict[sku]]
            outstr += '\multirow{17}{*}{'
            outstr += '{:.3f}'.format(all_acc)
            outstr += '}'
        outstr += f' & {time_dict[k][method]:.3f} \\\\'
        print(outstr)

In [30]:
for k in [1,2,3]:
    print(k, "-----overall-----")
    pretty_print_table(k=k)
    for sku in [f'cpu{num}' for num in [2, 4, 8, 16]]:
        print(f"-----{sku}-----")
        pretty_print_table(k=k, sku=sku)

1 -----overall-----
\textcolor{}{Variance} & 0.307 & 0.702 & 0.997 & 0.997 & \multirow{17}{*}{0.997} & 0.020 \\
\textcolor{}{fANOVA} & 0.961 & 0.969 & 0.978 & 0.988 &  & 0.040 \\
\textcolor{}{Chi2} & 0.545 & 0.904 & 0.969 & 0.997 &  & 0.062 \\
\textcolor{}{MIGain} & 0.963 & 0.961 & 0.989 & 0.991 &  & 2.403 \\
\textcolor{}{Pearson} & 0.961 & 0.969 & 0.978 & 0.990 &  & 0.030 \\
\textcolor{}{Lasso} & 0.835 & 0.962 & 0.978 & 0.997 &  & 0.049 \\
\textcolor{}{Ridge} & 0.940 & 0.987 & 0.994 & 0.994 &  & 0.051 \\
\textcolor{}{RFE Linear} & 0.961 & 0.970 & 0.990 & 0.992 &  & 1.128 \\
\textcolor{}{RFE DecTree} & 0.248 & 0.969 & 0.997 & 0.997 &  & 1.202 \\
\textcolor{}{RFE LogReg} & 0.789 & 0.965 & 0.993 & 0.997 &  & 18.936 \\
\textcolor{}{Fw SFS Linear} & 0.687 & 0.936 & 0.937 & 0.963 &  & 580.069 \\
\textcolor{}{Fw SFS DecTree} & 0.687 & 0.936 & 0.937 & 0.963 &  & 722.072 \\
\textcolor{}{Fw SFS LogReg} & 0.961 & 0.960 & 0.963 & 0.963 &  & 1829.737 \\
\textcolor{}{Bw SFS Linear} & 0.248 & 0.969 